In [14]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import mean_squared_error

train_df = pd.read_csv('data/train_feats.csv')
#train_df.head()

dev_df = pd.read_csv('data/dev_feats.csv')
#dev_df.head()

test_df = pd.read_csv('data/val_feats.csv')
#test_df.head()

train_df.head(10)

,hist_transactions_count,hist_most_frequent_merchant_cat,hist_most_frequent_subsector,hist_most_frequent_city,hist_most_frequent_state,hist_min_month_lag,hist_max_purchase_amount,new_transactions_count,new_most_frequent_merchant_cat,new_most_frequent_subsector,...,first_active_month_others,feature_1_2,feature_1_3,feature_1_5,feature_1_others,feature_2_1,feature_2_2,feature_2_3,feature_3_0,feature_3_1
0,382.0,278,37,25,7,-10.0,1.817983,8.0,307,7,...,1,0,0,0,1,1,0,0,1,0
1,39.0,879,29,149,20,-4.0,0.379389,5.0,2,20,...,1,1,0,0,0,1,0,0,1,0
2,188.0,705,33,226,16,-4.0,0.153180,16.0,80,37,...,1,0,0,0,1,1,0,0,1,0
3,19.0,307,19,57,5,-9.0,-0.342725,1.0,307,19,...,1,0,0,1,0,0,1,0,0,1
4,26.0,80,27,69,9,-3.0,-0.431381,8.0,80,27,...,1,1,0,0,0,1,0,0,1,0
5,101.0,307,19,17,22,-13.0,-0.255721,5.0,560,34,...,1,0,0,0,1,0,1,0,1,0
6,22.0,80,37,201,5,-3.0,-0.483944,7.0,80,27,...,1,0,1,0,0,1,0,0,0,1
7,138.0,705,33,116,9,-12.0,3.595754,1.0,606,17,...,1,0,1,0,0,0,0,1,0,1
8,249.0,80,37,69,9,-6.0,3.385383,10.0,80,34,...,1,0,0,0,1,1,0,0,1,0
9,236.0,705,33,117,23,-13.0,1501.922469,1.0,367,16,...,1,0,0,1,0,1,0,0,0,1


In [21]:
test_params = {"max_depth": [50,75,100],
              "learning_rate" : [0.003,0.001,0.03,0.01],
              "num_leaves": [100,300,900,1200],
             }
lgbm_model = lgb.LGBMRegressor()
model = GridSearchCV(cv = 5, estimator = lgbm_model , param_grid = test_params)
model.fit(train_df.drop('target',axis=1), train_df['target'])

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [50, 75, 100], 'learning_rate': [0.003, 0.001, 0.03, 0.01], 'num_leaves': [100, 300, 900, 1200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
print(model.best_params_) 

{'learning_rate': 0.03, 'max_depth': 50, 'num_leaves': 100}


In [28]:
params = {}
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mse'
params['learning_rate'] = 0.03
params['max_depth'] = 50
params['num_leaves'] = 100

d_train = lgb.Dataset(train_df, train_df['target'])

#train
lgbm_model = lgb.train(params, d_train, 1000) #perform 1000 iterations

In [30]:
#predict for dev
target_pred = lgbm_model.predict(dev_df)

#results for dev
RMSE = np.sqrt(mean_squared_error(target_pred, dev_df['target']))
print("RMSE for dev dataset: ", RMSE)

RMSE for dev dataset:  0.14261033663093942


In [32]:
#predict for test
target_pred = lgbm_model.predict(test_df)

#results for test
RMSE = np.sqrt(mean_squared_error(target_pred, test_df['target']))
print("RMSE for test dataset: ", RMSE)

RMSE for test dataset:  0.15736749388987978
